In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import numpy as np

In [3]:
# import the datasets
question = pd.read_csv(r'C:\\Users\\kfps86\\Downloads\\dataset\\question_metadata.csv')
answers = pd.read_csv(r'C:\\Users\\kfps86\\Downloads\\dataset\\answers_metadata.csv')
student = pd.read_csv(r'C:\\Users\\kfps86\\Downloads\\dataset\\student_metadata.csv')
subject = pd.read_csv(r'C:\\Users\\kfps86\\Downloads\\dataset\\subject_metadata.csv')
training = pd.read_csv(r'C:\\Users\\kfps86\\Downloads\\dataset\\training.csv')

In [4]:
# create a test_train split
from sklearn.model_selection import train_test_split
# np.random.seed(42)
train_set, test_set = train_test_split(training, test_size=0.2, random_state=42)

In [5]:
answers = answers.dropna(subset=['AnswerId']) 
# 7 values in AnswerId are na (out of 19834820), hence we are droppping those 7 values
answers['AnswerId'] = answers['AnswerId'].astype(int)

In [6]:
# merge the datasets
train_set = train_set.merge(answers , how='inner', on='AnswerId')
train_set = train_set.merge(student, how='inner', on='UserId')
train_set = train_set.merge(question, how='inner', on='QuestionId')

In [7]:
# drop nans
train_set.dropna(inplace=True)

In [8]:
train_set.head()

,QuestionId,UserId,AnswerId,IsCorrect,CorrectAnswer,AnswerValue,DateAnswered,Confidence,Gender,DateOfBirth,PremiumPupil,SubjectId
6,27185,40527,10797164,1,1,1,2019-03-18 22:03:00.000,100.0,1,2004-09-01 00:00:00.000,0.0,"[3, 49, 61, 171]"
20,27185,102085,11623933,1,1,1,2019-11-21 20:08:00.000,50.0,1,2005-09-01 00:00:00.000,0.0,"[3, 49, 61, 171]"
24,27185,86279,11606301,0,1,2,2019-02-09 12:45:00.000,50.0,2,2004-07-01 00:00:00.000,1.0,"[3, 49, 61, 171]"
26,27185,55310,15582540,1,1,1,2019-04-22 18:45:00.000,100.0,2,2003-09-01 00:00:00.000,0.0,"[3, 49, 61, 171]"
33,27185,34815,7797864,1,1,1,2019-11-20 18:28:00.000,100.0,2,2006-02-01 00:00:00.000,0.0,"[3, 49, 61, 171]"


In [9]:
# Data Cleaning
# need to change DateAnswered and DateOfBirth columns to datetime format
train_set['DateAnswered'] = pd.to_datetime(train_set['DateAnswered'], format='%Y-%m-%d %H:%M:%S.%f')
train_set['DateOfBirth'] = pd.to_datetime(train_set['DateOfBirth'], format='%Y-%m-%d %H:%M:%S.%f')

# change SubjectId to list format
train_set['SubjectId'] = train_set['SubjectId'].str.strip('[]').str.split(',')

# https://stackoverflow.com/questions/45312377/how-to-one-hot-encode-from-a-pandas-column-containing-a-list
from sklearn.preprocessing import MultiLabelBinarizer
# create a one hot encoding column for each category
# uses up a lot of RAM though
mlb = MultiLabelBinarizer(sparse_output=True)

train_set = train_set.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(train_set.pop('SubjectId')),
                index=train_set.index,
                columns=mlb.classes_))

# train_set['DateAnswered'] = train_set['DateAnswered'].values.astype('float')
# train_set['DateOfBirth'] = train_set['DateOfBirth'].values.astype('float')

In [25]:
# based on: https://github.com/ageron/handson-ml2/blob/master/02_end_to_end_machine_learning_project.ipynb
from sklearn.base import BaseEstimator, TransformerMixin

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
#     def __init__(self): # no *args or **kargs
    def fit(self, train_set, y=None):
        return self  # nothing else to do
    def transform(self, train_set):
        
        # cat => numerical value # 'total_answered',
        train_set['total_answered'] = train_set.groupby(['UserId'])['IsCorrect'].transform('count')

        # cat => numerical value # 'prop_correct',
        train_set['total_correct'] = train_set.groupby(['UserId'])['IsCorrect'].transform('sum')
        train_set['prop_correct'] = train_set['total_correct'] / train_set['total_answered']
#         train_set.drop('total_correct', inplace=True)

        # cat / numerical => numerical valueCMA
        train_set.sort_values(['UserId', 'DateAnswered'], inplace=True)
        CMA = train_set.groupby(['UserId']).IsCorrect.expanding().mean()
        train_set['CMA'] = CMA.reset_index(level=0, drop=True)

        # 'total_q_answered',
        train_set['total_q_answered'] = train_set.groupby(['QuestionId'])['QuestionId'].transform('count')

        # lvl2 - needs SubjectId first
        train_set['lvl2'] = 0
        for i in [' 101', ' 1156', ' 119', ' 149', ' 151', ' 32', ' 49', ' 692', ' 71']:
            if i in train_set.columns.tolist():
                i_int = (int(i[1:]))
                train_set['lvl2'] = train_set['lvl2'] + (train_set[i] * i_int)

        # CMA_correct_subject - need lvl2 first
        CMA_correct_subject = train_set.groupby(['UserId', 'lvl2']).IsCorrect.expanding().mean()
        train_set['CMA_correct_subject'] = CMA_correct_subject.reset_index(level=[0,1], drop=True)

        # 'holiday',
        train_set['holiday'] = 1
        train_set.loc[((train_set['DateAnswered'] < '2018-10-20') & (train_set['DateAnswered'] > '2018-09-03')) |
              ((train_set['DateAnswered'] > '2018-10-28') & (train_set['DateAnswered'] < '2018-12-20')) |
              ((train_set['DateAnswered'] > '2019-01-02') & (train_set['DateAnswered'] < '2019-02-16')) |
              ((train_set['DateAnswered'] > '2019-02-24') & (train_set['DateAnswered'] < '2019-04-06')) |
              ((train_set['DateAnswered'] > '2019-04-22') & (train_set['DateAnswered'] < '2019-05-25')) |
              ((train_set['DateAnswered'] > '2019-06-02') & (train_set['DateAnswered'] < '2019-07-25')) |

              ((train_set['DateAnswered'] > '2019-09-01') & (train_set['DateAnswered'] < '2019-10-19')) |
              ((train_set['DateAnswered'] > '2019-10-27') & (train_set['DateAnswered'] < '2019-12-20')) |
              ((train_set['DateAnswered'] > '2020-01-05') & (train_set['DateAnswered'] < '2020-02-15')) |
              ((train_set['DateAnswered'] > '2020-02-23') & (train_set['DateAnswered'] < '2020-04-03')) |
              ((train_set['DateAnswered'] > '2020-04-19') & (train_set['DateAnswered'] < '2020-05-23')) |
              ((train_set['DateAnswered'] > '2020-05-31') & (train_set['DateAnswered'] < '2020-07-23')) 
              ,'holiday'] = 0

        train_set['help'] = 0
        
        # 'unique_day',
        unique_student_train = pd.DataFrame(data=train_set['UserId'].unique(), columns=['UserId'])
        unique_student_train['unique_day'] = 0
        for i in range(len(unique_student_train)):
                unique_student_train.iloc[i, 1] =  len(train_set.loc[train_set['UserId']==unique_student_train.iloc[i, 0]]['DateAnswered'].dt.normalize().unique())
        train_set = train_set.merge(unique_student_train, how='inner', on='UserId')
        del unique_student_train
        import gc
        gc.collect()

        # 'yr2',
        train_set['yr2'] = 1
        train_set.loc[(train_set['DateAnswered'] < '2019-09-01'), 'yr2'] = 0

        # 'age',
        train_set['age'] = train_set['DateAnswered'] - train_set['DateOfBirth'] 

        # 'term',
        train_set['term'] = 6

        train_set.loc[((train_set['DateAnswered'] >= '2018-09-04') & (train_set['DateAnswered'] < '2018-10-29')) |
                      ((train_set['DateAnswered'] >= '2019-09-02') & (train_set['DateAnswered'] < '2019-10-28')),
                      'term'] = 1

        train_set.loc[((train_set['DateAnswered'] >= '2018-10-29') & (train_set['DateAnswered'] < '2019-01-03')) |
                      ((train_set['DateAnswered'] >= '2019-10-28') & (train_set['DateAnswered'] < '2020-01-06')),
                      'term'] = 2

        train_set.loc[((train_set['DateAnswered'] >= '2019-01-03') & (train_set['DateAnswered'] < '2019-02-25')) |
                      ((train_set['DateAnswered'] >= '2020-01-06') & (train_set['DateAnswered'] < '2020-02-24')),
                      'term'] = 3

        train_set.loc[((train_set['DateAnswered'] >= '2019-02-25') & (train_set['DateAnswered'] < '2019-04-23')) |
                      ((train_set['DateAnswered'] >= '2020-02-24') & (train_set['DateAnswered'] < '2020-04-20')),
                      'term'] = 4

        train_set.loc[((train_set['DateAnswered'] >= '2019-04-23') & (train_set['DateAnswered'] < '2019-06-03')) |
                      ((train_set['DateAnswered'] >= '2020-04-20') & (train_set['DateAnswered'] < '2020-06-01')),
                      'term'] = 5

        # 'time',
        train_set['time'] = 4
        train_set.loc[(train_set['DateAnswered'].dt.strftime("%H:%M:%S") >= '08:00:00') &
                      (train_set['DateAnswered'].dt.strftime("%H:%M:%S") < '12:00:00')
                       , 'time'] = 1

        train_set.loc[(train_set['DateAnswered'].dt.strftime("%H:%M:%S") >= '12:00:00') &
                      (train_set['DateAnswered'].dt.strftime("%H:%M:%S") < '16:00:00')
                       , 'time'] = 2

        train_set.loc[(train_set['DateAnswered'].dt.strftime("%H:%M:%S") >= '16:00:00') &
                      (train_set['DateAnswered'].dt.strftime("%H:%M:%S") < '20:00:00')
                       , 'time'] = 3

        # 'is_weekend',
        train_set['is_weekend'] = 0
        train_set.loc[train_set['DateAnswered'].dt.dayofweek > 4, 'is_weekend'] = 1

        # 'last_answered', adds repeat as well
        train_set.sort_values(['UserId', 'DateAnswered'], inplace=True)
        train_set['last_answered'] = train_set['DateAnswered'] - datetime.datetime.strptime('2018-09-01 00:00:00', '%Y-%m-%d %H:%M:%S')
        train_set['repeat'] = (train_set['UserId']==train_set['UserId'].shift(1))
        train_set.loc[train_set['repeat'] == True, 'last_answered'] = train_set['DateAnswered'].diff()
        
        # get rid of values
        train_set.drop('IsCorrect', axis=1, inplace=True)
        train_set.drop('DateAnswered', axis=1, inplace=True) # added
        train_set.drop('DateOfBirth', axis=1, inplace=True) # added
        # think issue is here
#         train_set['DateAnswered'] = train_set['DateAnswered'].values.astype('float')
#         train_set['DateOfBirth'] = train_set['DateOfBirth'].values.astype('float')
            
        return train_set

attr_adder = CombinedAttributesAdder()
training_extra_attribs = attr_adder.transform(train_set)

<ipython-input-25-d2c21bca1d9d>:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  train_set['total_answered'] = train_set.groupby(['UserId'])['IsCorrect'].transform('count')
<ipython-input-25-d2c21bca1d9d>:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  train_set['total_correct'] = train_set.groupby(['UserId'])['IsCorrect'].transform('sum')
<ipython-input-25-d2c21bca1d9d>:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newfra

In [11]:
# prep data for ML algos
# they use strat_train_set - think should do this based on confidence value
# getting equal missing values
IsCorrect = train_set.drop('IsCorrect', axis=1)
IsCorrect_labels = train_set['IsCorrect'].copy()
AnswerValue = train_set.drop('AnswerValue', axis=1)
AnswerValue_labels = train_set['AnswerValue'].copy()

In [15]:
train_set['DateAnswered_float'] = train_set['DateAnswered'].values.astype('float')
train_set['DateOfBirth_float'] = train_set['DateOfBirth'].values.astype('float')
train_set

<ipython-input-15-13b299ff3824>:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  train_set['DateAnswered_float'] = train_set['DateAnswered'].values.astype('float')
<ipython-input-15-13b299ff3824>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  train_set['DateOfBirth_float'] = train_set['DateOfBirth'].values.astype('float')


,QuestionId,UserId,AnswerId,IsCorrect,CorrectAnswer,AnswerValue,DateAnswered,Confidence,Gender,DateOfBirth,...,93,94,95,96,97,98,99,3,DateAnswered_float,DateOfBirth_float
6,27185,40527,10797164,1,1,1,2019-03-18 22:03:00,100.0,1,2004-09-01,...,0,0,0,0,0,0,0,1,1.552947e+18,1.093997e+18
20,27185,102085,11623933,1,1,1,2019-11-21 20:08:00,50.0,1,2005-09-01,...,0,0,0,0,0,0,0,1,1.574367e+18,1.125533e+18
24,27185,86279,11606301,0,1,2,2019-02-09 12:45:00,50.0,2,2004-07-01,...,0,0,0,0,0,0,0,1,1.549716e+18,1.088640e+18
26,27185,55310,15582540,1,1,1,2019-04-22 18:45:00,100.0,2,2003-09-01,...,0,0,0,0,0,0,0,1,1.555959e+18,1.062374e+18
33,27185,34815,7797864,1,1,1,2019-11-20 18:28:00,100.0,2,2006-02-01,...,0,0,0,0,0,0,0,1,1.574274e+18,1.138752e+18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12692074,3543,52636,8100523,1,2,2,2020-04-22 12:57:00,100.0,2,2006-12-01,...,0,0,0,0,0,0,0,1,1.587560e+18,1.164931e+18
12692077,3543,55302,12832011,1,2,2,2020-04-21 11:13:00,100.0,1,2007-03-01,...,0,0,0,0,0,0,0,1,1.587468e+18,1.172707e+18
12692079,3543,4938,7281923,0,2,4,2020-04-17 12:45:00,0.0,1,2006-11-01,...,0,0,0,0,0,0,0,1,1.587128e+18,1.162339e+18
12692080,3543,16245,18876238,0,2,4,2020-04-20 07:18:00,50.0,2,2007-01-01,...,0,0,0,0,0,0,0,1,1.587367e+18,1.167610e+18


In [29]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer


cat_attribs = ['QuestionId',
 'UserId',
 'AnswerId',
 'CorrectAnswer',
 'AnswerValue',
 'Confidence',
 'Gender',
 'PremiumPupil']

num_attribs = list(IsCorrect.drop(cat_attribs, axis=1))
num_attribs.remove('DateAnswered')
num_attribs.remove('DateOfBirth')

# all_attribs = cat_attribs + num_attribs
# all_attribs.append('IsCorrect')

all_attribs = list(train_set.drop(['DateAnswered_float', 'DateOfBirth_float'], axis=1))

# cat_attribs = ['QuestionId', 'UserId', 'AnswerId','CorrectAnswer', 'AnswerValue', 'Confidence', 'Gender', 'PremiumPupil', 'time', 'term']
# training_num = training_extra_attribs.drop(cat_attribs, axis=1)
# training_num['unique_day'] = 0
# num_attribs = list(training_num)


num_pipeline = Pipeline([
#         ('imputer', SimpleImputer(strategy="median")),
#         ('attribs_adder', CombinedAttributesAdder()),
        ('std_scaler', StandardScaler(with_mean=False)),
    ])

# all_pipeline = Pipeline([
#         ('attribs_adder', CombinedAttributesAdder()),
#         ('scale values', StandardScaler(with_mean=False)),
#     ])


full_pipeline = ColumnTransformer([
        ('all', CombinedAttributesAdder(), all_attribs),
        ('num', num_pipeline, num_attribs),
        ('cat', OneHotEncoder(), cat_attribs), # trying this as an extra but this should now work
    ])

# IsCorrect_prepared = full_pipeline.fit_transform(train_set)

IsCorrect_prepared = full_pipeline.fit_transform(train_set)
# IsCorrect_prepared = num_pipeline.fit_transform()

<ipython-input-25-d2c21bca1d9d>:67: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  train_set['yr2'] = 1
<ipython-input-25-d2c21bca1d9d>:71: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  train_set['age'] = train_set['DateAnswered'] - train_set['DateOfBirth']
<ipython-input-25-d2c21bca1d9d>:74: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  train_set['term'] = 6
<ipython-input-25-d2c21bca1d9d>:97: Performan

TypeError: float() argument must be a string or a number, not 'Timedelta'

In [ ]:
# still got error TypeError: float() argument must be a string or a number, not 'Timedelta'

IsCorrect_prepared
# this should now work - stay until finished running just in case

In [ ]:
# note this actually has run successfully thank god
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(C=1.0, penalty='l1', solver='liblinear')
log_reg.fit(IsCorrect_prepared, IsCorrect_labels)
log_reg_predictions = log_reg.predict(IsCorrect_prepared) # will give you 0 or 1 as the class
log_reg_predictions

In [ ]:
from sklearn.metrics import mean_squared_error
log_reg_mse = mean_squared_error(IsCorrect_labels, log_reg_predictions)
log_reg_rmse = np.sqrt(log_reg_mse)
log_reg_rmse

In [ ]:
# confusion matrix

# from https://colab.research.google.com/github/ageron/handson-ml2/blob/master/03_classification.ipynb#scrollTo=oWMedzLJ47vb 
from sklearn.model_selection import cross_val_predict

#y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3)

from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(IsCorrect_labels, log_reg_predictions)

In [ ]:
# accuracy
from sklearn.metrics import accuracy_score
acc = accuracy_score(IsCorrect_labels, log_reg_predictions, normalize=False)
acc

In [ ]:
# check this with sensitivity & specicivity
# F1-score
from sklearn.metrics import f1_score
f1 = f1_score(IsCorrect_labels, log_reg_predictions)
f1

In [ ]:
# sensitivity

In [ ]:
# specificity - should work may need to just get values from corr - matrix
from sklearn.metrics import classification_report
target_names = [0, 1]
print(classification_report(IsCorrect_labels, log_reg_predictions, target_names=target_names))

In [ ]:
# gridcv search

In [16]:
# split columns into numerical and categorical
cat_attribs = ['QuestionId', 'UserId', 'AnswerId','CorrectAnswer', 'AnswerValue', 'Confidence', 'Gender', 'PremiumPupil', 'time', 'term']
training_num = training_extra_attribs.drop(cat_attribs, axis=1)
training_num['unique_day'] = 0
num_attribs = list(training_num)
# all_attribs = list(train_set)

In [160]:
unique_student_train = pd.DataFrame(data=train_set['UserId'].unique(), columns=['UserId'])
print('2')
unique_student_train['unique_day'] = 0

2


In [20]:
['QuestionId',
 'UserId',
 'AnswerId',
 'CorrectAnswer',
 'AnswerValue',
 'Confidence',
 'Gender',
 'PremiumPupil']


train_set['DateAnswered'] = train_set['DateAnswered'].values.astype('float')

['QuestionId',
 'UserId',
 'AnswerId',
 'CorrectAnswer',
 'AnswerValue',
 'DateAnswered',
 'Confidence',
 'Gender',
 'DateOfBirth',
 'PremiumPupil',
 ' 100',
 ' 101',
 ' 102',
 ' 103',
 ' 104',
 ' 105',
 ' 1059',
 ' 106',
 ' 107',
 ' 1077',
 ' 1078',
 ' 1079',
 ' 108',
 ' 1080',
 ' 1081',
 ' 1082',
 ' 109',
 ' 110',
 ' 111',
 ' 112',
 ' 113',
 ' 114',
 ' 115',
 ' 1156',
 ' 1157',
 ' 1158',
 ' 1159',
 ' 116',
 ' 1160',
 ' 1161',
 ' 1162',
 ' 1163',
 ' 1164',
 ' 1165',
 ' 1167',
 ' 1169',
 ' 117',
 ' 1171',
 ' 1174',
 ' 1175',
 ' 1176',
 ' 1179',
 ' 118',
 ' 1180',
 ' 1181',
 ' 1182',
 ' 1184',
 ' 1185',
 ' 1186',
 ' 1187',
 ' 1188',
 ' 119',
 ' 1203',
 ' 1208',
 ' 1209',
 ' 1210',
 ' 1212',
 ' 1213',
 ' 1214',
 ' 1215',
 ' 1218',
 ' 1263',
 ' 1265',
 ' 1266',
 ' 141',
 ' 144',
 ' 146',
 ' 149',
 ' 152',
 ' 153',
 ' 154',
 ' 156',
 ' 157',
 ' 158',
 ' 159',
 ' 160',
 ' 163',
 ' 1636',
 ' 164',
 ' 1642',
 ' 1647',
 ' 1648',
 ' 1649',
 ' 165',
 ' 1650',
 ' 1651',
 ' 166',
 ' 167',
 ' 1676'

In [36]:
IsCorrect.head()

,QuestionId,UserId,AnswerId,CorrectAnswer,AnswerValue,DateAnswered,Confidence,Gender,DateOfBirth,PremiumPupil,...,91,92,93,94,95,96,97,98,99,3
6,27185,40527,10797164,1,1,2019-03-18 22:03:00,100.0,1,2004-09-01,0.0,...,0,0,0,0,0,0,0,0,0,1
20,27185,102085,11623933,1,1,2019-11-21 20:08:00,50.0,1,2005-09-01,0.0,...,0,0,0,0,0,0,0,0,0,1
24,27185,86279,11606301,1,2,2019-02-09 12:45:00,50.0,2,2004-07-01,1.0,...,0,0,0,0,0,0,0,0,0,1
26,27185,55310,15582540,1,1,2019-04-22 18:45:00,100.0,2,2003-09-01,0.0,...,0,0,0,0,0,0,0,0,0,1
33,27185,34815,7797864,1,1,2019-11-20 18:28:00,100.0,2,2006-02-01,0.0,...,0,0,0,0,0,0,0,0,0,1


In [79]:
train_set['DateAnswered_float'] = train_set['DateAnswered'].values.astype('float')
train_set['DateOfBirth_float'] = train_set['DateOfBirth'].values.astype('float')
train_set

,QuestionId,UserId,AnswerId,IsCorrect,CorrectAnswer,AnswerValue,DateAnswered,Confidence,Gender,DateOfBirth,...,93,94,95,96,97,98,99,3,DateAnswered_float,DateOfBirth_float
6,27185,40527,10797164,1,1,1,2019-03-18 22:03:00,100.0,1,2004-09-01,...,0,0,0,0,0,0,0,1,1.552947e+18,1.093997e+18
20,27185,102085,11623933,1,1,1,2019-11-21 20:08:00,50.0,1,2005-09-01,...,0,0,0,0,0,0,0,1,1.574367e+18,1.125533e+18
24,27185,86279,11606301,0,1,2,2019-02-09 12:45:00,50.0,2,2004-07-01,...,0,0,0,0,0,0,0,1,1.549716e+18,1.088640e+18
26,27185,55310,15582540,1,1,1,2019-04-22 18:45:00,100.0,2,2003-09-01,...,0,0,0,0,0,0,0,1,1.555959e+18,1.062374e+18
33,27185,34815,7797864,1,1,1,2019-11-20 18:28:00,100.0,2,2006-02-01,...,0,0,0,0,0,0,0,1,1.574274e+18,1.138752e+18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12692074,3543,52636,8100523,1,2,2,2020-04-22 12:57:00,100.0,2,2006-12-01,...,0,0,0,0,0,0,0,1,1.587560e+18,1.164931e+18
12692077,3543,55302,12832011,1,2,2,2020-04-21 11:13:00,100.0,1,2007-03-01,...,0,0,0,0,0,0,0,1,1.587468e+18,1.172707e+18
12692079,3543,4938,7281923,0,2,4,2020-04-17 12:45:00,0.0,1,2006-11-01,...,0,0,0,0,0,0,0,1,1.587128e+18,1.162339e+18
12692080,3543,16245,18876238,0,2,4,2020-04-20 07:18:00,50.0,2,2007-01-01,...,0,0,0,0,0,0,0,1,1.587367e+18,1.167610e+18


In [76]:
train_set

,QuestionId,UserId,AnswerId,IsCorrect,CorrectAnswer,AnswerValue,DateAnswered,Confidence,Gender,DateOfBirth,...,91,92,93,94,95,96,97,98,99,3
6,27185,40527,10797164,1,1,1,2019-03-18 22:03:00,100.0,1,2004-09-01,...,0,0,0,0,0,0,0,0,0,1
20,27185,102085,11623933,1,1,1,2019-11-21 20:08:00,50.0,1,2005-09-01,...,0,0,0,0,0,0,0,0,0,1
24,27185,86279,11606301,0,1,2,2019-02-09 12:45:00,50.0,2,2004-07-01,...,0,0,0,0,0,0,0,0,0,1
26,27185,55310,15582540,1,1,1,2019-04-22 18:45:00,100.0,2,2003-09-01,...,0,0,0,0,0,0,0,0,0,1
33,27185,34815,7797864,1,1,1,2019-11-20 18:28:00,100.0,2,2006-02-01,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12692074,3543,52636,8100523,1,2,2,2020-04-22 12:57:00,100.0,2,2006-12-01,...,0,0,0,0,0,0,0,0,0,1
12692077,3543,55302,12832011,1,2,2,2020-04-21 11:13:00,100.0,1,2007-03-01,...,0,0,0,0,0,0,0,0,0,1
12692079,3543,4938,7281923,0,2,4,2020-04-17 12:45:00,0.0,1,2006-11-01,...,0,0,0,0,0,0,0,0,0,1
12692080,3543,16245,18876238,0,2,4,2020-04-20 07:18:00,50.0,2,2007-01-01,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
num_attribs

TypeError: float() argument must be a string or a number, not 'Timedelta'

In [58]:
cat_attribs = ['QuestionId',
 'UserId',
 'AnswerId',
 'CorrectAnswer',
 'AnswerValue',
 'Confidence',
 'Gender',
 'PremiumPupil']

num_attribs = list(IsCorrect.drop(cat_attribs, axis=1))

# all_attribs = cat_attribs + num_attribs
# all_attribs.append('IsCorrect')



In [59]:
all_attribs

['QuestionId',
 'UserId',
 'AnswerId',
 'CorrectAnswer',
 'AnswerValue',
 'Confidence',
 'Gender',
 'PremiumPupil',
 'DateAnswered',
 'DateOfBirth',
 ' 100',
 ' 101',
 ' 102',
 ' 103',
 ' 104',
 ' 105',
 ' 1059',
 ' 106',
 ' 107',
 ' 1077',
 ' 1078',
 ' 1079',
 ' 108',
 ' 1080',
 ' 1081',
 ' 1082',
 ' 109',
 ' 110',
 ' 111',
 ' 112',
 ' 113',
 ' 114',
 ' 115',
 ' 1156',
 ' 1157',
 ' 1158',
 ' 1159',
 ' 116',
 ' 1160',
 ' 1161',
 ' 1162',
 ' 1163',
 ' 1164',
 ' 1165',
 ' 1167',
 ' 1169',
 ' 117',
 ' 1171',
 ' 1174',
 ' 1175',
 ' 1176',
 ' 1179',
 ' 118',
 ' 1180',
 ' 1181',
 ' 1182',
 ' 1184',
 ' 1185',
 ' 1186',
 ' 1187',
 ' 1188',
 ' 119',
 ' 1203',
 ' 1208',
 ' 1209',
 ' 1210',
 ' 1212',
 ' 1213',
 ' 1214',
 ' 1215',
 ' 1218',
 ' 1263',
 ' 1265',
 ' 1266',
 ' 141',
 ' 144',
 ' 146',
 ' 149',
 ' 152',
 ' 153',
 ' 154',
 ' 156',
 ' 157',
 ' 158',
 ' 159',
 ' 160',
 ' 163',
 ' 1636',
 ' 164',
 ' 1642',
 ' 1647',
 ' 1648',
 ' 1649',
 ' 165',
 ' 1650',
 ' 1651',
 ' 166',
 ' 167',
 ' 1676'

In [43]:
IsCorrect_prepared

<537824x572746 sparse matrix of type '<class 'numpy.float64'>'
	with 169414560 stored elements in Compressed Sparse Row format>

# 20.01 Lucy
start again above
unsure what is happeneing with the key error
try to run book code https://github.com/ageron/handson-ml2/blob/master/02_end_to_end_machine_learning_project.ipynb
see if that helps
once pipeline works then we're looking a lot better

think we're gonna use random forest but maybe do some more research

In [ ]:
# # add the columns to IsCorrect
# IsCorrect['total_answered'] = ''
# IsCorrect['prop_correct'] = ''
# IsCorrect['CMA'] = ''
# IsCorrect['total_q_answered'] = ''
# IsCorrect['lvl2'] = ''
# IsCorrect['CMA_correct_subject'] = ''
# IsCorrect['holiday'] = ''
# IsCorrect['unique_day                                                                                                                                    '] = ''
# IsCorrect['yr2'] = ''
# IsCorrect['age'] = ''
# IsCorrect['term'] = ''
# IsCorrect['time'] = ''
# IsCorrect['is_weekend'] = ''
# IsCorrect['last_answered'] = ''
# IsCorrect['repeat'] = ''

In [140]:
# 'total_answered',
#         train_set['total_answered'] = train_set.groupby(['UserId'])['IsCorrect'].transform('count')

#         # 'prop_correct',
#         train_set['total_correct'] = train_set.groupby(['UserId'])['IsCorrect'].transform('sum')
#         train_set['prop_correct'] = train_set['total_correct'] / train_set['total_answered']
# #         train_set.drop('total_correct', inplace=True)
                
#         # CMA
#         train_set.sort_values(['UserId', 'DateAnswered'], inplace=True)
#         CMA = train_set.groupby(['UserId']).IsCorrect.expanding().mean()
#         train_set['CMA'] = CMA.reset_index(level=0, drop=True)
        
#         # 'total_q_answered',
#         train_set['total_q_answered'] = train_set.groupby(['QuestionId'])['QuestionId'].transform('count')

# # lvl2 - needs SubjectId first
# train_set['lvl2'] = 0
# for i in [' 101', ' 1156', ' 119', ' 149', ' 151', ' 32', ' 49', ' 692', ' 71']:
#     if i in train_set.columns.tolist():
#         i_int = (int(i[1:]))
#         train_set['lvl2'] = train_set['lvl2'] + (train_set[i] * i_int)
        
# # CMA_correct_subject - need lvl2 first
# CMA_correct_subject = train_set.groupby(['UserId', 'lvl2']).IsCorrect.expanding().mean()
# train_set['CMA_correct_subject'] = CMA_correct_subject.reset_index(level=[0,1], drop=True)

# # 'holiday',
# train_set['holiday'] = 1
# train_set.loc[((train_set['DateAnswered'] < '2018-10-20') & (train_set['DateAnswered'] > '2018-09-03')) |
#       ((train_set['DateAnswered'] > '2018-10-28') & (train_set['DateAnswered'] < '2018-12-20')) |
#       ((train_set['DateAnswered'] > '2019-01-02') & (train_set['DateAnswered'] < '2019-02-16')) |
#       ((train_set['DateAnswered'] > '2019-02-24') & (train_set['DateAnswered'] < '2019-04-06')) |
#       ((train_set['DateAnswered'] > '2019-04-22') & (train_set['DateAnswered'] < '2019-05-25')) |
#       ((train_set['DateAnswered'] > '2019-06-02') & (train_set['DateAnswered'] < '2019-07-25')) |

#       ((train_set['DateAnswered'] > '2019-09-01') & (train_set['DateAnswered'] < '2019-10-19')) |
#       ((train_set['DateAnswered'] > '2019-10-27') & (train_set['DateAnswered'] < '2019-12-20')) |
#       ((train_set['DateAnswered'] > '2020-01-05') & (train_set['DateAnswered'] < '2020-02-15')) |
#       ((train_set['DateAnswered'] > '2020-02-23') & (train_set['DateAnswered'] < '2020-04-03')) |
#       ((train_set['DateAnswered'] > '2020-04-19') & (train_set['DateAnswered'] < '2020-05-23')) |
#       ((train_set['DateAnswered'] > '2020-05-31') & (train_set['DateAnswered'] < '2020-07-23')) 
#       ,'holiday'] = 0

# 'unique_day',
unique_student_train = pd.DataFrame(data=train_set['UserId'].unique(), columns=['UserId'])
unique_student_train['unique_day'] = 0
for i in range(len(unique_student_train)):
        unique_student_train.iloc[i, 1] =  len(train_set.loc[train_set['UserId']==unique_student_train.iloc[i, 0]]['DateAnswered'].dt.normalize().unique())
train_set = train_set.merge(unique_student_train, how='inner', on='UserId')
del unique_student_train
import gc
gc.collect()

# 'yr2',
train_set['yr2'] = 1
train_set.loc[(train_set['DateAnswered'] < '2019-09-01'), 'yr2'] = 0

# 'age',
train_set['age'] = train_set['DateAnswered'] - train_set['DateOfBirth'] 

# 'term',
train_set['term'] = 6

train_set.loc[((train_set['DateAnswered'] >= '2018-09-04') & (train_set['DateAnswered'] < '2018-10-29')) |
              ((train_set['DateAnswered'] >= '2019-09-02') & (train_set['DateAnswered'] < '2019-10-28')),
              'term'] = 1

train_set.loc[((train_set['DateAnswered'] >= '2018-10-29') & (train_set['DateAnswered'] < '2019-01-03')) |
              ((train_set['DateAnswered'] >= '2019-10-28') & (train_set['DateAnswered'] < '2020-01-06')),
              'term'] = 2

train_set.loc[((train_set['DateAnswered'] >= '2019-01-03') & (train_set['DateAnswered'] < '2019-02-25')) |
              ((train_set['DateAnswered'] >= '2020-01-06') & (train_set['DateAnswered'] < '2020-02-24')),
              'term'] = 3

train_set.loc[((train_set['DateAnswered'] >= '2019-02-25') & (train_set['DateAnswered'] < '2019-04-23')) |
              ((train_set['DateAnswered'] >= '2020-02-24') & (train_set['DateAnswered'] < '2020-04-20')),
              'term'] = 4

train_set.loc[((train_set['DateAnswered'] >= '2019-04-23') & (train_set['DateAnswered'] < '2019-06-03')) |
              ((train_set['DateAnswered'] >= '2020-04-20') & (train_set['DateAnswered'] < '2020-06-01')),
              'term'] = 5

# 'time',
train_set['time'] = 4
train_set.loc[(train_set['DateAnswered'].dt.strftime("%H:%M:%S") >= '08:00:00') &
              (train_set['DateAnswered'].dt.strftime("%H:%M:%S") < '12:00:00')
               , 'time'] = 1

train_set.loc[(train_set['DateAnswered'].dt.strftime("%H:%M:%S") >= '12:00:00') &
              (train_set['DateAnswered'].dt.strftime("%H:%M:%S") < '16:00:00')
               , 'time'] = 2

train_set.loc[(train_set['DateAnswered'].dt.strftime("%H:%M:%S") >= '16:00:00') &
              (train_set['DateAnswered'].dt.strftime("%H:%M:%S") < '20:00:00')
               , 'time'] = 3

# 'is_weekend',
train_set['is_weekend'] = 0
train_set.loc[train_set['DateAnswered'].dt.dayofweek > 4, 'is_weekend'] = 1

# 'last_answered', adds repeat as well
train_set.sort_values(['UserId', 'DateAnswered'], inplace=True)
train_set['last_answered'] = train_set['DateAnswered'] - datetime.datetime.strptime('2018-09-01 00:00:00', '%Y-%m-%d %H:%M:%S')
train_set['repeat'] = (train_set['UserId']==train_set['UserId'].shift(1))
train_set.loc[train_set['repeat'] == True, 'last_answered'] = train_set['DateAnswered'].diff() 

KeyboardInterrupt: 

In [ ]:
# source: lvl 2 book pg 59
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(m_training, test_size=0.2, random_state=42)

In [33]:
# based on: https://github.com/ageron/handson-ml2/blob/master/02_end_to_end_machine_learning_project.ipynb
from sklearn.base import BaseEstimator, TransformerMixin

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
#     def __init__(self): # no *args or **kargs
    def fit(self, train_set, y=None):
        return self  # nothing else to do
    def transform(self, train_set):
        
        # 'total_answered',
        train_set['total_answered'] = train_set.groupby(['UserId'])['IsCorrect'].transform('count')

        # 'prop_correct',
        train_set['total_correct'] = train_set.groupby(['UserId'])['IsCorrect'].transform('sum')
        train_set['prop_correct'] = train_set['total_correct'] / train_set['total_answered']
#         train_set.drop('total_correct', inplace=True)

        # CMA
        train_set.sort_values(['UserId', 'DateAnswered'], inplace=True)
        CMA = train_set.groupby(['UserId']).IsCorrect.expanding().mean()
        train_set['CMA'] = CMA.reset_index(level=0, drop=True)

        # 'total_q_answered',
        train_set['total_q_answered'] = train_set.groupby(['QuestionId'])['QuestionId'].transform('count')

        # lvl2 - needs SubjectId first
        train_set['lvl2'] = 0
        for i in [' 101', ' 1156', ' 119', ' 149', ' 151', ' 32', ' 49', ' 692', ' 71']:
            if i in train_set.columns.tolist():
                i_int = (int(i[1:]))
                train_set['lvl2'] = train_set['lvl2'] + (train_set[i] * i_int)

        # CMA_correct_subject - need lvl2 first
        CMA_correct_subject = train_set.groupby(['UserId', 'lvl2']).IsCorrect.expanding().mean()
        train_set['CMA_correct_subject'] = CMA_correct_subject.reset_index(level=[0,1], drop=True)

        # 'holiday',
        train_set['holiday'] = 1
        train_set.loc[((train_set['DateAnswered'] < '2018-10-20') & (train_set['DateAnswered'] > '2018-09-03')) |
              ((train_set['DateAnswered'] > '2018-10-28') & (train_set['DateAnswered'] < '2018-12-20')) |
              ((train_set['DateAnswered'] > '2019-01-02') & (train_set['DateAnswered'] < '2019-02-16')) |
              ((train_set['DateAnswered'] > '2019-02-24') & (train_set['DateAnswered'] < '2019-04-06')) |
              ((train_set['DateAnswered'] > '2019-04-22') & (train_set['DateAnswered'] < '2019-05-25')) |
              ((train_set['DateAnswered'] > '2019-06-02') & (train_set['DateAnswered'] < '2019-07-25')) |

              ((train_set['DateAnswered'] > '2019-09-01') & (train_set['DateAnswered'] < '2019-10-19')) |
              ((train_set['DateAnswered'] > '2019-10-27') & (train_set['DateAnswered'] < '2019-12-20')) |
              ((train_set['DateAnswered'] > '2020-01-05') & (train_set['DateAnswered'] < '2020-02-15')) |
              ((train_set['DateAnswered'] > '2020-02-23') & (train_set['DateAnswered'] < '2020-04-03')) |
              ((train_set['DateAnswered'] > '2020-04-19') & (train_set['DateAnswered'] < '2020-05-23')) |
              ((train_set['DateAnswered'] > '2020-05-31') & (train_set['DateAnswered'] < '2020-07-23')) 
              ,'holiday'] = 0

        train_set['help'] = 0
        
        # 'unique_day',
        unique_student_train = pd.DataFrame(data=train_set['UserId'].unique(), columns=['UserId'])
        unique_student_train['unique_day'] = 0
        for i in range(len(unique_student_train)):
                unique_student_train.iloc[i, 1] =  len(train_set.loc[train_set['UserId']==unique_student_train.iloc[i, 0]]['DateAnswered'].dt.normalize().unique())
        train_set = train_set.merge(unique_student_train, how='inner', on='UserId')
        del unique_student_train
        import gc
        gc.collect()

        # 'yr2',
        train_set['yr2'] = 1
        train_set.loc[(train_set['DateAnswered'] < '2019-09-01'), 'yr2'] = 0

        # 'age',
        train_set['age'] = train_set['DateAnswered'] - train_set['DateOfBirth'] 

        # 'term',
        train_set['term'] = 6

        train_set.loc[((train_set['DateAnswered'] >= '2018-09-04') & (train_set['DateAnswered'] < '2018-10-29')) |
                      ((train_set['DateAnswered'] >= '2019-09-02') & (train_set['DateAnswered'] < '2019-10-28')),
                      'term'] = 1

        train_set.loc[((train_set['DateAnswered'] >= '2018-10-29') & (train_set['DateAnswered'] < '2019-01-03')) |
                      ((train_set['DateAnswered'] >= '2019-10-28') & (train_set['DateAnswered'] < '2020-01-06')),
                      'term'] = 2

        train_set.loc[((train_set['DateAnswered'] >= '2019-01-03') & (train_set['DateAnswered'] < '2019-02-25')) |
                      ((train_set['DateAnswered'] >= '2020-01-06') & (train_set['DateAnswered'] < '2020-02-24')),
                      'term'] = 3

        train_set.loc[((train_set['DateAnswered'] >= '2019-02-25') & (train_set['DateAnswered'] < '2019-04-23')) |
                      ((train_set['DateAnswered'] >= '2020-02-24') & (train_set['DateAnswered'] < '2020-04-20')),
                      'term'] = 4

        train_set.loc[((train_set['DateAnswered'] >= '2019-04-23') & (train_set['DateAnswered'] < '2019-06-03')) |
                      ((train_set['DateAnswered'] >= '2020-04-20') & (train_set['DateAnswered'] < '2020-06-01')),
                      'term'] = 5

        # 'time',
        train_set['time'] = 4
        train_set.loc[(train_set['DateAnswered'].dt.strftime("%H:%M:%S") >= '08:00:00') &
                      (train_set['DateAnswered'].dt.strftime("%H:%M:%S") < '12:00:00')
                       , 'time'] = 1

        train_set.loc[(train_set['DateAnswered'].dt.strftime("%H:%M:%S") >= '12:00:00') &
                      (train_set['DateAnswered'].dt.strftime("%H:%M:%S") < '16:00:00')
                       , 'time'] = 2

        train_set.loc[(train_set['DateAnswered'].dt.strftime("%H:%M:%S") >= '16:00:00') &
                      (train_set['DateAnswered'].dt.strftime("%H:%M:%S") < '20:00:00')
                       , 'time'] = 3

        # 'is_weekend',
        train_set['is_weekend'] = 0
        train_set.loc[train_set['DateAnswered'].dt.dayofweek > 4, 'is_weekend'] = 1

        # 'last_answered', adds repeat as well
        train_set.sort_values(['UserId', 'DateAnswered'], inplace=True)
        train_set['last_answered'] = train_set['DateAnswered'] - datetime.datetime.strptime('2018-09-01 00:00:00', '%Y-%m-%d %H:%M:%S')
        train_set['repeat'] = (train_set['UserId']==train_set['UserId'].shift(1))
        train_set.loc[train_set['repeat'] == True, 'last_answered'] = train_set['DateAnswered'].diff()
        
        return train_set

attr_adder = CombinedAttributesAdder()
# training_extra_attribs = attr_adder.transform(train_set)
